# A simple itemCF Baseline, score:0.1169(phase0-2) by青禹小生


https://tianchi.aliyun.com/forum/postDetail?postId=103530

In [1]:
#!/usr/bin/env python  
# -*- coding:utf-8 -*-  
  
import pandas as pd  
from tqdm import tqdm  
from collections import defaultdict  
import math  
  
def get_sim_item(df, user_col, item_col, use_iif=False):  
    user_item_ = df.groupby(user_col)[item_col].agg(set).reset_index()  
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))  
  
    sim_item = {}  
    item_cnt = defaultdict(int)  
    for user, items in tqdm(user_item_dict.items()):  
        for i in items:  
            item_cnt[i] += 1  
            sim_item.setdefault(i, {})  
            for relate_item in items:  
                if i == relate_item:  
                    continue  
                sim_item[i].setdefault(relate_item, 0)  
                if not use_iif:  
                    sim_item[i][relate_item] += 1  
                else:  
                    sim_item[i][relate_item] += 1 / math.log(1 + len(items))  
    sim_item_corr = sim_item.copy()  
    for i, related_items in tqdm(sim_item.items()):  
        for j, cij in related_items.items():  
            sim_item_corr[i][j] = cij/math.sqrt(item_cnt[i]*item_cnt[j])  
  
    return sim_item_corr, user_item_dict  
  

def recommend(sim_item_corr, user_item_dict, user_id, top_k, item_num):  
    rank = {}  
    interacted_items = user_item_dict[user_id]  
    for i in interacted_items:  
        for j, wij in sorted(sim_item_corr[i].items(), reverse=True)[0:top_k]:  
            if j not in interacted_items:  
                rank.setdefault(j, 0)  
                rank[j] += wij  
    return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]  
  

# fill user to 50 items  
def get_predict(df, pred_col, top_fill):  
    top_fill = [int(t) for t in top_fill.split(',')]  
    scores = [-1 * i for i in range(1, len(top_fill) + 1)]  
    ids = list(df['user_id'].unique())  
    fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])  
    fill_df.sort_values('user_id', inplace=True)  
    fill_df['item_id'] = top_fill * len(ids)  
    fill_df[pred_col] = scores * len(ids)  
    df = df.append(fill_df)  
    df.sort_values(pred_col, ascending=False, inplace=True)  
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')  
    df['rank'] = df.groupby('user_id')[pred_col].rank(method='first', ascending=False)  
    df = df[df['rank'] <= 50]  
    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',', expand=True).reset_index()  
    return df  
  

if __name__ == "__main__":  
    now_phase = 1  
    train_path = 'underexpose_train'  
    test_path = 'underexpose_test'  
    recom_item = []  
  
    whole_click = pd.DataFrame()  
    for c in range(now_phase + 1):  
        print('phase:', c)  
        click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
        click_test = pd.read_csv(test_path + '/underexpose_test_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'])  
  
        all_click = click_train.append(click_test)  
        whole_click = whole_click.append(all_click)  
        item_sim_list, user_item = get_sim_item(all_click, 'user_id', 'item_id', use_iif=True)  
  
        for i in tqdm(click_test['user_id'].unique()):  
            rank_item = recommend(item_sim_list, user_item, i, 500, 50)  
            for j in rank_item:  
                recom_item.append([i, j[0], j[1]])  
    # find most popular items  
    top50_click = whole_click['item_id'].value_counts().index[:50].values  
    top50_click = ','.join([str(i) for i in top50_click])  
  
    recom_df = pd.DataFrame(recom_item, columns=['user_id', 'item_id', 'sim'])  
    result = get_predict(recom_df, 'sim', top50_click)  
    result.to_csv('baseline.csv', index=False, header=None)

phase: 0


FileNotFoundError: [Errno 2] File underexpose_train/underexpose_train_click-0.csv does not exist: 'underexpose_train/underexpose_train_click-0.csv'

# 开源版本 0.2
https://mp.weixin.qq.com/s/GbnhlATPGHwVsmF069VpHw


"该赛道的数据集强调电商推荐系统的公平性，尤其是流量较少的广大中小商家所面临的“有好货缺无人问津”的困境。数据横跨十余天，中间还穿插了某次全网促销活动，涵盖了一些商品从上新时无人问津、到逐渐成为高潜力爆款的历程。欲获胜的队伍需格外关注曝光不足的商品上的推荐准确度，需探索“如何抵消掉历史点击数据的选择性偏差以便避免只推爆款”、“如何注意数据分布随时间的变化以便及时发现高潜力冷门好货”、“如何利用多模态图文商品信息来辅助商品冷启动”等重要课题。"--解题关键

       首先感谢青禹小生的开源，本文是在其基础上进行更多的细化， 也将对更多优化方向和建模思路进行简单介绍。

In [ ]:
whole_click = whole_click.drop_duplicates(subset=['user_id','item_id','time'],keep='last')
whole_click = whole_click.sort_values('time')

## 关联商品打分

In [6]:
def get_sim_item(df_, user_col, item_col, use_iif=False):

    df = df_.copy()
    user_item_ = df.groupby(user_col)[item_col].agg(list).reset_index()
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))

    user_time_ = df.groupby(user_col)['time'].agg(list).reset_index() # 引入时间因素
    user_time_dict = dict(zip(user_time_[user_col], user_time_['time']))

    sim_item = {}  
    item_cnt = defaultdict(int)  # 商品被点击次数
    for user, items in tqdm(user_item_dict.items()):  
        for loc1, item in enumerate(items):  
            item_cnt[item] += 1  
            sim_item.setdefault(item, {})  
            for loc2, relate_item in enumerate(items):  
                if item == relate_item:  
                    continue  
                t1 = user_time_dict[user][loc1] # 点击时间提取
                t2 = user_time_dict[user][loc2]
                sim_item[item].setdefault(relate_item, 0)  
                if not use_iif:  
                    if loc1-loc2>0:
                        sim_item[item][relate_item] += 1 * 0.7 * (0.8**(loc1-loc2-1)) * (1 - (t1 - t2) * 10000) / math.log(1 + len(items)) # 逆向
                    else:
                        sim_item[item][relate_item] += 1 * 1.0 * (0.8**(loc2-loc1-1)) * (1 - (t2 - t1) * 10000) / math.log(1 + len(items)) # 正向
                else:  
                    sim_item[item][relate_item] += 1 / math.log(1 + len(items))

    sim_item_corr = sim_item.copy() # 引入AB的各种被点击次数  
    for i, related_items in tqdm(sim_item.items()):  
        for j, cij in related_items.items():  
            sim_item_corr[i][j] = cij / ((item_cnt[i] * item_cnt[j]) ** 0.2)

    return sim_item_corr, user_item_dict

这里在原有基础上考虑了两点因素，关联位置因素和关联时间因素。强关联的发生是有向的、有位置的和有时间的。

比如我们先买了手机，那下一次买手机壳的关联，和先买手机壳再买手机的关联，这两种很明显，A到B大于B到A，这是有向性；

我先买了手机，然后买了手机壳，又买了耳机，很明显，手机和手机壳的关联性大于手机与耳机的关联性，这是位置性；

那么如果再加上时间这层因素，时间相隔越远的关联性肯定是不高的。

这三点因素就可以组成我们的优化思路，有向性打分__位置打分__时间打分，得到最终关联打分。

## 交互行为打分

In [3]:
def recommend(sim_item_corr, user_item_dict, user_id, top_k, item_num):

    rank = {}  
    interacted_items = user_item_dict[user_id] 
    interacted_items = interacted_items[::-1]
    for loc, i in enumerate(interacted_items):  
        for j, wij in sorted(sim_item_corr[i].items(), reverse=True)[0:top_k]:  
            if j not in interacted_items:  
                rank.setdefault(j, 0)  
                rank[j] += wij * (0.7**loc) 
                
    return sorted(rank.items(), key=lambda d: d[1], reverse=True)[:item_num]

这里的优化也很符合我们的主观认识，距离下次点击月近的行为，相关性越接近，所有可以根据位置远近考虑重要性，添加权重因子。当然还可以添加时间权重因子。



## 优化方向

切勿陷入思维定势，也许我的优化方向和baseline会使大家产生一个误区。就像之前安泰杯的比赛，决赛中评委也说到了baseline，很大程度影响的大家的思维方向。没有说baseline不好，而这只能作为无数解题思路中的一小部分，一个分支。不是沿着这个分支走下去，而是去创造更多分支。__这就如同多路召回，之将其当作融合的一部分罢了__。

目前只是从关联的角度解决问题，这个角度可以做的更细。也可以考虑其它方向，目前大家做的还都是起点，而不是终点。向量召回、模型召回，以及后面的排序都需要进一步尝试。



## 建模思路

没有相关经验的同学可能会问
### 正负样本怎么来，
那其实很简单，需要我们去构造label。这里分为两步：

#### 样本提取。
我们线下验证的时候，一般是用户最后一次点击进行验证，会进行召回50个商品，然后观察召回率。这样的50个商品及对应的用户就是样本数据。

#### 样本打标。
召回的50个商品中是最后一次点击的商品labael是1，反之为0。

这样我们就能得到训练集，测试集构造方式一样，只不过需要去预测其label，最后将label的概率进行排序，top50就是最终建模得到的结果。



最后我们就
#### 可以像一般的二分类问题进行解题了。
为了防止数据泄露，构造特征时一定要用历史的点击行为进行构造。

根据历史数据，给定上一次点击数据情况下。用户点击过某商品则label=1,其他皆为负样本。负样本可抽样。

# A Simple Recall Method based on Network-based Inference by BruceQD
https://tianchi.aliyun.com/forum/postDetail?spm=5176.12281915.0.0.50cc29c82GYcQg&postId=104936

开源的recall思路


召回阶段很重要，可尝试多种方式、方法。下面给出一种基于二分网络的召回方法，该方法来自于电子科技大学周涛教授的代表作“Bipartite network projection and personal recommendation”。其核心思想在于下图，建议感兴趣的同学去读读这篇论文。

思路还很多, local random walk、common neighbors based on bayesian之类的, 基于里面分享的论文，可以去查查引用它的文献


核心代码如下：

In [4]:
def get_sim_item(df, user_col, item_col, use_iif=False):  
    user_item_ = df.groupby(user_col)[item_col].agg(set).reset_index()  
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))  
    
    item_user_ = whole_click.groupby(item_col)[user_col].agg(set).reset_index()  
    item_user_dict = dict(zip(item_user_[item_col], item_user_[user_col]))    

    item_cnt = defaultdict(int)  
    for user, items in tqdm(user_item_dict.items()):  
        for i in items:  
            item_cnt[i] += 1  

    sim_item = {}

    for item, users in tqdm(item_user_dict.items()):
    
        sim_item.setdefault(item, {}) 
    
        for u in users:
        
            tmp_len = len(user_item_dict[u])
        
            for relate_item in user_item_dict[u]:
                sim_item[item].setdefault(relate_item, 0)
                sim_item[item][relate_item] += 1/ (math.log(len(users)+1) * math.log(tmp_len+1))
            

            
            
    return sim_item, user_item_dict  

其他部分可参考青羽小生开源的baseline。

## 召回阶段的效果决定了排序后的天花板，所以要多尝试些思路提高召回效果，尽可能保证真正点击的商品在召回列表中。之后，再做重排序（第二阶段），甚至精排序（第三阶段）。

## Bipartite network projection and personal recommendation

### abstract
One-mode projecting is extensively used to compress bipartite networks. Since one-mode projection is always less informative than the bipartite representation, a proper weighting method is required to better retain the original information. In this article, inspired by the network-based resource-allocation dynamics, we raisea weighting method which can be directly applied in extracting the hidden information of networks, with remarkably better performance than the widely used global ranking method as well as collaborative filtering. This work not only provides a creditable method for compressing bipartite networks, but also highlights apossible way for the better solution of a long-standing challenge in modern information science: How to do a personal recommendation.

单模投影被广泛用于压缩两方网络。 由于单模投影的信息量总是少于二分法表示，因此需要一种适当的加权方法以更好地保留原始信息。 在本文中，基于网络资源分配动态的启发，我们提出了一种加权方法，该方法可直接用于提取网络的隐藏信息，其性能要比广泛使用的全局排名方法和协作过滤明显好。 不仅提供了一种可靠的压缩双向网络的方法，而且还强调了一种可行的方法，可以更好地解决现代信息科学中长期存在的挑战：如何进行个人推荐。

### Introduction
A particular class of networks is 
#### the bipartite networks, whose nodes are divided into two sets XandY, and only the con-nection between two nodes in different sets is allowedas illustrated in Fig.1a
![](img/fig01.png)

for the convenience of directly showing the relations among a particular set of nodes, the bipartite network is usu-ally compressed by one-mode projecting. The one-mode pro-jection onto X--X projection for short means a network containing only X nodes, where two Xnodes are connectedwhen they have at least one common neighboring Y node.

The simplest way is to project the bipartite   network   onto   an   unweighted   network, without taking into account the frequency that a collaboration has been repeatedn, the loss of information is obvious.

### Bipartite network projection
Without loss of generality, we discuss how to determinethe edge weight in Xprojection, where the weight w_ij can be considered as the importance of node i in j’s sense, and it is generally not equal to w_ji. 

For example, in the book projec-tion of a customer-book opinion network, the weight wij be-tween two books i and j contributes to the strength of book i recommendation to a customer provided he has bought book j. 

为了得出wij的解析表达式，我们回到二分表示法。 由于双向网络本身未加权，因此，任意Xnode中的资源应平均分配给Y中的邻居。 类似地，anyYnode中的资源应平均分配到其Xneighbors。 如图2a所示，最初为这三个X节点分配了权重x，y和z。 资源分配过程包括两个步骤： 首先从XtoY，然后回到X。 每个步骤之后的资源量分别在图2b和2c中标记。 将这两个步骤合并为一个


![](img/img1.png)